# STIX2 Tooling Tutorial

STIX2 tooling described here is all available via open source repositories. Most are OASIS Open, some are not.

[OASIS CTI Repositories](https://github.com/oasis-open?q=cti)

## Requirements

```pip install stix2 stix2-patterns stix2-viz stix2-validator```

## Getting data via TAXII

Use [python-stix2](https://github.com/oasis-open/cti-python-stix2) to connect to a TAXII server:

In [1]:
# import the stix2 library and the taxii2client libraries
from stix2 import TAXIICollectionSource, Filter
from taxii2client import Collection

# establish TAXII2 Collection instance
collection = Collection("https://limo.anomali.com/api/v1/taxii2/feeds/collections/107/", user="guest", password="guest")

# supply the TAXII2 collection to TAXIICollection
tc_source = TAXIICollectionSource(collection)

# build your filter -- kinda seems broken now
# f1 = Filter("type","=", "indicator")

#retrieve the STIX objects
results = tc_source.query()

print("Retrieving...")
print("Got {} results".format(len(results)))

Retrieving...
Got 1001 results


## Making sure you can use the data

Next we can take the indicators that come back and actually do something with them. In this case, we'll inspect what types of objects and comparisons are contained in the pattern.

In [2]:
# Import the pattern inspector
from stix2patterns.pattern import Pattern

# In this case, just get the first indicator
indicator = results[0]

# Print some basic information
print("Name: " + indicator.name)
print("Pattern: {}\n".format(indicator.pattern))

parsed_pattern = Pattern(indicator.pattern)

print("Pattern inspection: " + str(parsed_pattern.inspect()))

Name: phish_url: http://www.srbiohealth.com/London1/Eruku/nD/index.php
Pattern: [url:value = 'http://www.srbiohealth.com/London1/Eruku/nD/index.php']

Pattern inspection: pattern_data(comparisons={'url': [(['value'], '=', "'http://www.srbiohealth.com/London1/Eruku/nD/index.php'")]}, observation_ops=set(), qualifiers=set())


## Creating a Sighting

While actually matching the pattern is out scope for this tutorial, let's assume that the pattern matched and create a Sighting with the result.

In [3]:
# Import the Sighting object to use it
from stix2 import Sighting
from datetime import datetime

sighting = Sighting(
    sighting_of_ref=indicator.id,
    first_seen=datetime.now(),
    last_seen=datetime.now(),
    count=1
)

print("Sighting:\n{}".format(str(sighting)))

Sighting:
{
    "type": "sighting",
    "id": "sighting--99c034df-19b0-41cf-9a94-cb55023be5ee",
    "created": "2017-12-07T16:09:40.017Z",
    "modified": "2017-12-07T16:09:40.017Z",
    "first_seen": "2017-12-07T17:09:40.017261Z",
    "last_seen": "2017-12-07T17:09:40.017268Z",
    "count": 1,
    "sighting_of_ref": "indicator--644c3b1b-7133-4e9c-a28b-519c12e66a5d"
}


## Updating the Sighting

What if we saw a couple more sightings? Let's update it!

In [20]:
updated_sighting = sighting.new_version(
    count=5,
    last_seen=datetime.now()
)

print("Sighting:\n{}".format(str(updated_sighting)))

Sighting:
{
    "type": "sighting",
    "id": "sighting--99c034df-19b0-41cf-9a94-cb55023be5ee",
    "created": "2017-12-07T16:09:40.017Z",
    "modified": "2017-12-07T16:15:45.601Z",
    "first_seen": "2017-12-07T17:09:40.017261Z",
    "last_seen": "2017-12-07T17:15:45.60144Z",
    "count": 5,
    "sighting_of_ref": "indicator--644c3b1b-7133-4e9c-a28b-519c12e66a5d"
}


## Giving the indicator context

What if we know what malware instance the indicator detects? Well, let's create it and add the relationship.

In [5]:
from stix2 import Malware, Relationship

# Create the malware...just some basic details for now
wannacry = Malware(name="WannaCry", description="No, I don't really", labels=['ransomware'])

# Then relate the indicator to it
rel = Relationship(indicator, 'indicates', wannacry)

print("Malware:\n{}".format(str(wannacry)))
print("Relationship:\n{}".format(str(rel)))


Malware:
{
    "type": "malware",
    "id": "malware--210be9b7-8c41-4b4d-942e-5888015db6b5",
    "created": "2017-12-07T16:09:44.837Z",
    "modified": "2017-12-07T16:09:44.837Z",
    "name": "WannaCry",
    "description": "No, I don't really",
    "labels": [
        "ransomware"
    ]
}
Relationship:
{
    "type": "relationship",
    "id": "relationship--488536e9-e5a5-44e5-84d6-e20e0af929dd",
    "created": "2017-12-07T16:09:44.837Z",
    "modified": "2017-12-07T16:09:44.837Z",
    "relationship_type": "indicates",
    "source_ref": "indicator--644c3b1b-7133-4e9c-a28b-519c12e66a5d",
    "target_ref": "malware--210be9b7-8c41-4b4d-942e-5888015db6b5"
}


## What does this stuff look like?

Next, we can use the visualizer to see what the relationship diagram looks like.

In [6]:
from stix2 import Bundle
import stix2viz # This is Jupyter notebook-specific

bundle = Bundle(indicator, wannacry, rel, sighting)

stix2viz.display(str(bundle))

## Sharing it Back

You can also take the sighting and share it back via TAXII.

In [23]:
collection = Collection('https://ubertaxii.com/scratch/collections/4a26ee4a-924c-4d5e-8519-5d87efe7f6a8/', user='ubertaxii', password='certain_players_want_a_pony')

status = collection.add_objects(str(Bundle(sighting)))

In [24]:
print("Status: " + status.status)
print("Successes: {}".format(status.success_count))
print("Failures: {}".format(status.failure_count))
print("Pending: {}".format(status.pending_count))

Status: complete
Successes: 1
Failures: 0
Pending: 0
